In [1]:
import numpy as np
from plyfile import PlyData, PlyElement
import open3d as o3d
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def convert_3dgs_colors_to_rgb(vertex_data):
    """3DGS 색상 값을 uint8 (0~255)로 변환."""
    f_dc_0 = vertex_data['f_dc_0']
    f_dc_1 = vertex_data['f_dc_1']
    f_dc_2 = vertex_data['f_dc_2']
    
    # 각 채널별 정규화
    f_dc_0_norm = (f_dc_0 - f_dc_0.min()) / (f_dc_0.max() - f_dc_0.min())
    f_dc_1_norm = (f_dc_1 - f_dc_1.min()) / (f_dc_1.max() - f_dc_1.min())
    f_dc_2_norm = (f_dc_2 - f_dc_2.min()) / (f_dc_2.max() - f_dc_2.min())
    
    red = (f_dc_0_norm * 255).astype(np.uint8)
    green = (f_dc_1_norm * 255).astype(np.uint8)
    blue = (f_dc_2_norm * 255).astype(np.uint8)
    
    print(f"Converted R range: {red.min()} - {red.max()}")
    print(f"Converted G range: {green.min()} - {green.max()}")
    print(f"Converted B range: {blue.min()} - {blue.max()}")
    
    return red, green, blue

def merge_and_process_point_clouds(scene_id, data_root):
    """
    3DGS와 Mesh Point Cloud를 병합하고, 중복 점 제거, Outlier 제거 후 PLY로 저장.
    
    Args:
        scene_id (str): Scene ID (예: scene0011_00).
        data_root (str): 데이터 루트 경로.
    """
    # 경로 설정
    mesh_ply = f"{data_root}/{scene_id}/{scene_id}_vh_clean_2.ply"
    point_3dgs_ply = f"{data_root}/{scene_id}/point_cloud.ply"
    output_ply = f"{data_root}/{scene_id}/merged_point_cloud.ply"

    # 출력 디렉터리 생성
    os.makedirs(os.path.dirname(output_ply), exist_ok=True)

    # 1. Mesh Point Cloud 로드
    pcd_mesh = o3d.io.read_point_cloud(mesh_ply)
    points_mesh = np.asarray(pcd_mesh.points)
    colors_mesh = np.asarray(pcd_mesh.colors) * 255  # 0~1 → 0~255로 변환
    colors_mesh = colors_mesh.astype(np.uint8)

    # 2. 3DGS Point Cloud 로드 및 색상 변환
    with open(point_3dgs_ply, 'rb') as f:
        ply_data_3dgs = PlyData.read(f)
    vertex_data_3dgs = ply_data_3dgs['vertex']
    
    points_3dgs = np.stack([vertex_data_3dgs['x'], vertex_data_3dgs['y'], vertex_data_3dgs['z']], axis=-1)
    red, green, blue = convert_3dgs_colors_to_rgb(vertex_data_3dgs)
    colors_3dgs = np.stack([red, green, blue], axis=-1)

    # 3. 중복 점 제거 (Mesh Point Cloud의 점 제거)
    # 3DGS 점과 Mesh 점 간의 중복 확인
    tree = o3d.geometry.KDTreeFlann(pcd_mesh)
    indices_to_remove = []
    
    for i, point in enumerate(points_3dgs):
        [k, idx, _] = tree.search_knn_vector_3d(point, 1)
        if k > 0 and np.linalg.norm(points_mesh[idx[0]] - point) < 1e-6:  # 중복 점 판단 기준
            indices_to_remove.append(idx[0])
    
    # 중복 점 제거 (Mesh에서 제거)
    indices_to_keep = np.setdiff1d(np.arange(len(points_mesh)), indices_to_remove)
    points_mesh = points_mesh[indices_to_keep]
    colors_mesh = colors_mesh[indices_to_keep]

    # 4. 병합
    points_merged = np.vstack((points_mesh, points_3dgs))
    colors_merged = np.vstack((colors_mesh, colors_3dgs))

    # 5. Open3D Point Cloud 객체 생성
    merged_pcd = o3d.geometry.PointCloud()
    merged_pcd.points = o3d.utility.Vector3dVector(points_merged)
    merged_pcd.colors = o3d.utility.Vector3dVector(colors_merged / 255.0)  # 0~255 → 0~1로 변환 (Open3D 요구사항)

    # 6. Outlier 제거 (Statistical Outlier Removal)
    merged_pcd, _ = merged_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

    # 7. PLY 파일로 저장
    points_final = np.asarray(merged_pcd.points)
    colors_final = (np.asarray(merged_pcd.colors) * 255).astype(np.uint8)  # 0~1 → 0~255로 변환
    
    vertex_array = np.array(
        [(points_final[i, 0], points_final[i, 1], points_final[i, 2],
          colors_final[i, 0], colors_final[i, 1], colors_final[i, 2])
         for i in range(len(points_final))],
        dtype=[
            ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
            ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')
        ]
    )
    
    vertex_element = PlyElement.describe(vertex_array, 'vertex')
    new_ply = PlyData([vertex_element], text=False)3
    new_ply.write(output_ply)
    print(f"Merged and processed point cloud saved to {output_ply}")

# 사용 예제
data_root = "/home/knuvi/Desktop/song/gaussian-point-sampler/dataset"
scene_id = "scene0011_00"
merge_and_process_point_clouds(scene_id, data_root)

Converted R range: 0 - 255
Converted G range: 0 - 255
Converted B range: 0 - 255
Merged and processed point cloud saved to /home/knuvi/Desktop/song/gaussian-point-sampler/dataset/scene0011_00/merged_point_cloud.ply
